## Báo cáo bài tập lớn môn Học máy

Giảng viên: Trần Quốc Long

Lớp môn học: INT3405_1

Sinh viên: Đào Ngọc Việt Anh

MSSV: 18020163

## 1. Mô tả bài toán
* Hiện nay, xử lý những nội dung độc hại và gây chia rẽ là một thách thức đối với bất kỳ trang web nào.Quora muốn giải quyết vấn đề này trực tiếp để giữ cho nền tảng của họ trở thành một nơi mà người dùng có thể cảm thấy an toàn khi chia sẻ kiến thức của họ với cộng đồng.

* Vậy mục tiêu của bài toán này là xác định được những câu hỏi có nội dung nhạy cảm để có thể loại bỏ chúng. Đó là những câu hỏi không mang tính chất đóng góp, thiếu chân thành, thậm chí để đả kích một cá nhân, tập thể hay tổ chức nào đó.
* Input: Câu hỏi từ Quora ở dạng text

* Output: giá trị 0 hoặc 1 (0: câu hỏi chân thành; 1: câu hỏi không chân thành)

## 2. Phân tích dữ liệu

### Import những thư viện cần thiết


In [ ]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np 
import pandas as pd 

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

from string import punctuation 

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import f1_score, classification_report
from xgboost import XGBClassifier
import lightgbm as lgb

In [ ]:
df_train = pd.read_csv("../input/quora-insincere-questions-classification/train.csv")
df_test = pd.read_csv("../input/quora-insincere-questions-classification/test.csv", index_col='qid')
df_train.tail()

Dữ liệu gồm 3 cột:

qid: mã số câu hỏi

question_text: câu hỏi

target: nhãn dữ liệu (0 hoặc 1)

In [ ]:
df_train.info()

In [ ]:
df_test.info()

Tập train gồm có 1306122 câu hỏi, tập test gồm có 375806 câu hỏi

In [ ]:
#Ta biểu diễn đồ thị tỉ lệ số câu hỏi chân thành và số câu hỏi không chân thành trong tập train
import collections

fig, ax = plt.subplots(1,1, figsize=(7,7))
explode=(0,0.1)
labels ='sincere','insincere'
colors = ['#00ff00', '#ff0000']

counts = list(dict(collections.Counter(list(df_train.target))).values()) 
ax.pie(counts , explode=explode, labels=labels, colors=colors, autopct='%1.1f%%',shadow=True, startangle=90)

### Nhận xét về dữ liệu

Dựa vào biểu đồ trên ta có thể thấy, số lượng câu hỏi gán nhãn chân thành chiếm phần lớn với tỷ lệ 93.8% còn những câu hỏi không chân thành chỉ chiếm 6.12% 

=> Tập dữ liệu bị mất cân bằng, vì thế nên khi đánh giá chất lượng mô hình ta không nên lựa chọn độ chính xác(accuracy) làm chỉ số đánh giá mô hình. Thay vào đó ta có thể sử dụng các metric thay thế như: F1_score, Recall,..
 

### Khảo sát về tần suất các từ xuất hiện trong tập dữ liệu:

Word clouds: Một từ càng xuất hiện nhiều trong văn bản thì từ đó càng lớn và đậm nét hơn trong Word clouds.

In [ ]:
from wordcloud import WordCloud, STOPWORDS
stopwords = set(STOPWORDS)

In [ ]:
print('Ảnh word cloud được tạo từ những câu hỏi chân thành:')
sincere_wordcloud = WordCloud(width=600, height=400, background_color ='white', min_font_size = 10).generate(str(df_train[df_train["target"] == 0]["question_text"]))
#Positive Word cloud
plt.figure(figsize=(15,6), facecolor=None)
plt.imshow(sincere_wordcloud)
plt.axis("off")
plt.tight_layout(pad=0)
plt.show();

In [ ]:
print('Ảnh word cloud được tạo từ những câu hỏi thiếu chân thành:')
insincere_wordcloud = WordCloud(width=600, height=400, background_color ='white', min_font_size = 10).generate(str(df_train[df_train["target"] == 1]["question_text"]))
#Positive Word cloud
plt.figure(figsize=(15,6), facecolor=None)
plt.imshow(insincere_wordcloud)
plt.axis("off")
plt.tight_layout(pad=0)
plt.show();

### Thống kê những từ phổ biến trong các loại câu hỏi

Stop Words: Là những từ thường được sử dụng ( chẳng hạn như 'the', 'an', 'a',..). Để tránh những từ này chiếm dung lượng trong cơ sở dữ liệu hoặc mất thời gian xử lí, chúng ta nên loại bỏ chúng.

In [ ]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
stop_words

In [ ]:
class Vocabulary(object):
    
    def __init__(self):
        self.vocab = {}
        self.STOPWORDS = set()
        self.STOPWORDS = set(stopwords.words('english'))
        
    def build_vocab(self, lines):
        for line in lines:
            for word in line.split(' '):
                word = word.lower()
                if (word in self.STOPWORDS):
                    continue
                if (word not in self.vocab):
                    self.vocab[word] = 0
                self.vocab[word] +=1 
    
    def generate_ngrams(text, n_gram=1):
        """arg: text, n_gram"""
        token = [token for token in text.lower().split(" ") if token != "" if token not in STOPWORDS]
        ngrams = zip(*[token[i:] for i in range(n_gram)])
        return [" ".join(ngram) for ngram in ngrams]
    
    def horizontal_bar_chart(df, color):
        trace = go.Bar(
            y=df["word"].values[::-1],
            x=df["wordcount"].values[::-1],
            showlegend=False,
            orientation = 'h',
            marker=dict(
            color=color,
            ),
        )
        return trace

In [ ]:
sincere_vocab = Vocabulary()
sincere_vocab.build_vocab(df_train[df_train['target'] == 0]['question_text'])
sincere_vocabulary = sorted(sincere_vocab.vocab.items(), reverse=True, key=lambda kv: kv[1])
    
df_sincere_vocab = pd.DataFrame(sincere_vocabulary, columns=['word_sincere', 'frequency'])
sns.barplot(y='word_sincere', x='frequency', data=df_sincere_vocab[:20])

In [ ]:
insincere_vocab = Vocabulary()
insincere_vocab.build_vocab(df_train[df_train['target'] == 1]['question_text'])
insincere_vocabulary = sorted(insincere_vocab.vocab.items(), reverse=True, key=lambda kv: kv[1])

df_insincere_vocab = pd.DataFrame(insincere_vocabulary, columns=['word_insincere', 'frequency'])
sns.barplot(y='word_insincere', x='frequency', data=df_insincere_vocab[:20])

### Nhận xét

Như chúng ta có thể thấy rõ ràng có một số từ nhất định (chửi thề, từ ngữ phân biệt đối xử dựa trên chủng tộc, nhân vật chính trị, v.v.) xuất hiện rất nhiều trong các câu thiếu chân thành.

### Xử lý dữ liệu

Sau khi kiểm tra xong dữ liệu, ta có thể thấy dữ liệu đang khá phức tạp và nhiễu nên ta cần làm một số việc sau:
* Chỉ giữ lại các ký tự chữ và số 
* Loại bỏ các stop word 
* Đưa các từ về đúng dạng gốc

**Tokenization** : (tách từ) là một trong những bước quan trọng nhất trong quá trình tiền xử lý văn bản. Nói một cách đơn giản, tokenization là quá trình tách một cụm từ, câu, đoạn văn, một hoặc nhiều tài liệu văn bản thành các đơn vị nhỏ hơn. Mỗi đơn vị nhỏ hơn này được gọi là Tokens.

**Stemming vs. lemmatization**: Với mục đích là giảm các dạng vô hướng của mỗi từ thành một cơ sở hoặc gốc chung.

In [ ]:
df_train = df_train[['question_text', 'target']]

def text_processing(local_df):
    """ return the dataframe with tokens stemmetized without numerical values & stopwords """
    stemmer = PorterStemmer()
    # Perform preprocessing
    local_df['txt_processed'] = local_df['question_text'].apply(lambda df: word_tokenize(df))
    local_df['txt_processed'] = local_df['txt_processed'].apply(lambda x: [item for item in x if item.isalpha()])
    local_df['txt_processed'] = local_df['txt_processed'].apply(lambda x: [item for item in x if item not in stop_words])
    local_df['txt_processed'] = local_df['txt_processed'].apply(lambda x: [stemmer.stem(item) for item in x])
    return local_df

In [ ]:
df_train = text_processing(df_train)
df_train.tail()

## 3. Training Model



Như đã nói ở trên, trong bài toán này ta không đánh giá độ chính xác của mô hình dựa trên Accuracy mà sẽ đánh giá dựa trên metric khác cụ thể là F1-score. 

In [ ]:
def get_fscore_matrix(fitted_clf, model_name):
    print(model_name, ' :')
    
    # get classes predictions for the classification report 
    y_train_pred, y_pred = fitted_clf.predict(X_train), fitted_clf.predict(X_test)
    print(classification_report(y_test, y_pred), '\n') # target_names=y
    
    # computes probabilities keep the ones for the positive outcome only      
    print(f'F1-score = {f1_score(y_test, y_pred):.2f}')

Đối với bài toán này, ta sử dụng mô hình Logistic Regression Mô hình này giống với Linear Regression ở khía cạnh đầu ra là số thực, và giống với PLA ở việc đầu ra bị chặn (trong đoạn 0 -> 1). Mặc dù trong tên có chứa từ "regression", tuy vậy Logistic Regression thường được sử dụng nhiều cho các bài toán classification. Do vậy, em lựa chọn Logistic Regression cho bài toán này.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer(analyzer="word", ngram_range=(1,4), max_df=0.9)
model = LogisticRegression(solver="saga", class_weight="balanced", C=0.5, max_iter=250, verbose=1, n_jobs=-1)

vectorize_model_pipeline = Pipeline([
    ('count_vectorizer', count_vectorizer),
    ('model', model)
])

Mặc dù đã xử lý dữ liệu ở phần phía trên, tuy vậy, dữ liệu đầu vào của ta vẫn là dạng Text (nó sạch hơn một chút thôi). Để sử dụng dữ liệu văn bản cho mô hình dự đoán, văn bản phải được phân tích cú pháp để loại bỏ một số từ nhất định - quá trình này được gọi là mã hóa . 

Sau đó, những từ này cần được mã hóa dưới dạng số nguyên hoặc giá trị dấu phẩy động, để sử dụng làm đầu vào trong thuật toán học máy. Quá trình này được gọi là trích xuất đặc trưng (hoặc vectơ hóa).

**CountVectorizer** được sử dụng để chuyển đổi một bộ sưu tập các tài liệu văn bản thành một vectơ có số lượng thuật ngữ / mã thông báo. Nó cũng cho phép xử lý trước dữ liệu văn bản trước khi tạo biểu diễn vectơ. Chức năng này làm cho nó trở thành một mô-đun biểu diễn tính năng rất linh hoạt cho văn bản.

Hay nói ngắn gọn là sử dụng **CountVectorizer** để chuyển dữ liệu đầu vào từ Text sang Vectơ


In [ ]:
# Convert a collection of text documents to string
df_train['str_processed'] = df_train['txt_processed'].apply(lambda x: " ".join(x))
df_train.head(2)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_train['str_processed'], df_train['target'], test_size = 0.2, 
                                                    stratify = df_train.target.values)


In [ ]:
pl_model = vectorize_model_pipeline.fit(X_train, y_train)
pl_model

In [ ]:
get_fscore_matrix(pl_model, 'Pipeline')

In [ ]:
df_test = text_processing(df_test)
df_test['str_processed'] = df_test['txt_processed'].apply(lambda x: " ".join(x))
df_test.head(2)

In [ ]:
y_pred_final = pl_model.predict(df_test['str_processed'])
y_pred_final

In [ ]:
df_submission = pd.DataFrame({"qid":df_test.index, "prediction":y_pred_final})
df_submission.head()

In [ ]:
df_submission.to_csv('submission.csv', index=False)

## 4. Đánh giá mô hình Logistic Regression

Logistic Regression là mô hình dễ sử dụng, và dễ hiểu, số điểm nó đạt được cũng tương đối cao:

Private Score: 0.62545

Public Score: 0.61680

Logistic Regression là một mô hình được sử dụng nhiều trong các bài toán Classification.

Mặc dù có tên là Regression, tức một mô hình cho fitting, Logistic Regression lại được sử dụng nhiều trong các bài toán Classification. Sau khi tìm được mô hình, việc xác định class y cho một điểm dữ liệu x được xác định bằng việc so sánh hai biểu thức xác suất: P(y = 1|x; w); P(y = 0|x; w) Nếu biểu thức thứ nhất lớn hơn thì ta kết luận điểm dữ liệu thuộc class 1, ngược lại thì nó thuộc class 0. Vì tổng hai biểu thức này luôn bằng 1 nên một cách gọn hơn, ta chỉ cần xác định xem P(y = 1|x; w) lớn hơn 0.5 hay không. Nếu có, class 1. Nếu không, class 0.

Logistic Regression đạt được số điểm khá tốt